In [1]:
import torch

def f(x):
    return torch.randn(x.size(0), x.size(1))

sm_randn = torch.jit.script(f)
print(sm_randn.graph)

graph(%x.1 : Tensor):
  %8 : NoneType = prim::Constant()
  %2 : int = prim::Constant[value=0]() # /tmp/ipykernel_2878822/2877097691.py:4:30
  %5 : int = prim::Constant[value=1]() # /tmp/ipykernel_2878822/2877097691.py:4:41
  %3 : int = aten::size(%x.1, %2) # /tmp/ipykernel_2878822/2877097691.py:4:23
  %6 : int = aten::size(%x.1, %5) # /tmp/ipykernel_2878822/2877097691.py:4:34
  %7 : int[] = prim::ListConstruct(%3, %6)
  %12 : Tensor = aten::randn(%7, %8, %8, %8, %8) # /tmp/ipykernel_2878822/2877097691.py:4:11
  return (%12)



In [1]:
import torch
import torch.nn as nn
from brt.runtime import log
from brt.app import RandScatter
from brt.router import GatherRouter


log.set_level("frontend", "INFO")
log.set_level("backend", "INFO")
log.set_level("ir", "INFO")


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = RandScatter(path_num=2)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)
        self.gather_router = GatherRouter()

    def forward(self, x):
        route_results = self.scatter_router(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1])
        return x


class MoEModel(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


moe_model = MoEModel()
x = torch.arange(0, 30, dtype=torch.float32).view(3, 10)
x = moe_model(x)
print(x)


tensor([[ -2.1933,   0.5259,   5.3248,  -2.4949,   0.6522,   3.0274,   0.7719,
          -0.4352,   5.2967,   1.8439],
        [ -4.0227,  10.2557,   7.6807,   7.3433,  -2.1963,  14.8702,   1.8967,
          -8.2479,  -7.2180,  -8.7040],
        [ -7.2936,  17.2094,  12.7483,  12.7952,  -4.7621,  24.6789,   1.9324,
         -14.0765, -13.6807, -15.8386]], grad_fn=<AliasBackward0>)


In [2]:
from brt.frontend import build_graph
from brt.backend.pytorch import model_to_script
from brt.frontend import symbolize

script_moe_model = torch.jit.script(symbolize(moe_model))
sm_graph = script_moe_model.graph
ir_moe_model = build_graph(moe_model)
model_script = model_to_script(ir_moe_model)
print(model_script)


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import brt


class MoEModel_model__moe(nn.Module):
    def __init__(self):
        super().__init__()
        self._scatter_router = brt.routers.app.rand.RandScatterRouter(dst_num=2)
        self._expert1 = torch.nn.modules.linear.Linear(in_features=10, out_features=10)
        self._expert2 = torch.nn.modules.linear.Linear(in_features=10, out_features=10)
        self._gather_router = brt.routers.router.GatherRouter(dst_num=2)
        self._mapping_ = {'_scatter_router': 'moe.scatter_router', '_expert1': 'moe.expert1', '_expert2': 'moe.expert2', '_gather_router': 'moe.gather_router'}

    def forward(self, x__1):
        _Constant2 = 0
        _Constant3 = 1
        _scatter_router = self._scatter_router(x__1)
        _aten____getitem__6 = _scatter_router[_Constant2]
        _aten____getitem__8 = _scatter_router[_Constant3]
        _expert1 = self._expert1(_aten____getitem__6)


In [3]:
import torch
import torch.nn as nn
from brt.runtime import log
from brt.router import ScatterRouter, GatherRouter


route_func = nn.Sequential(nn.Linear(10, 2), nn.ReLU())


class DynamicRouting(nn.Module):
    def __init__(self, dst_num):
        super().__init__()
        self.scatter_router_0 = ScatterRouter(
            dst_num=dst_num,
            route_func=route_func,
            route_method="threshold",
        )
        self.scatter_router_1 = ScatterRouter(
            dst_num=dst_num,
            route_func=route_func,
            route_method="threshold",
        )
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 20)
        self.expert3 = nn.Linear(10, 10)
        self.expert4 = nn.Linear(10, 20)
        self.gather_router_0 = GatherRouter(dst_num=dst_num)
        self.gather_router_1 = GatherRouter(dst_num=dst_num)

    def forward(self, x, y):
        route_results_x = self.scatter_router_0(x)
        route_results_y = self.scatter_router_1(y)
        x_0 = self.expert1(route_results_x[0])
        x_1 = self.expert2(route_results_x[1])
        y_0 = self.expert3(route_results_y[0])
        y_1 = self.expert4(route_results_y[1])
        x = self.gather_router_0([x_0, y_0])
        y = self.gather_router_1([x_1, y_1])
        return x, y


dy_model = DynamicRouting(2)

x = torch.randn((2, 10))
y = torch.randn((2, 10))

x, y = dy_model(x, y)

print(x.shape)
print(y.shape)


[2022-06-19 20:58:21] WARNING (brainstorm.routers/MainThread) threshold is not specified for Threshold route method, use default threshold=0.0
[2022-06-19 20:58:21] WARNING (brainstorm.routers/MainThread) threshold is not specified for Threshold route method, use default threshold=0.0
torch.Size([2, 10])
torch.Size([1, 20])


In [5]:
import torch
import torch.nn as nn
from brt.runtime import log
from brt.router import ScatterRouter, GatherRouter


route_func = nn.Sequential(nn.Linear(10, 2), nn.ReLU())


class DynamicRouting(nn.Module):
    def __init__(self, dst_num):
        super().__init__()
        self.scatter_router_0 = ScatterRouter(
            dst_num=dst_num,
            route_func=route_func,
            route_method="threshold",
            threshold=0,
            residual_dst=0,
        )
        self.scatter_router_1 = ScatterRouter(
            dst_num=dst_num,
            route_func=route_func,
            route_method="threshold",
            threshold=0,
            residual_dst=0,
        )
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 20)
        self.expert3 = nn.Linear(10, 10)
        self.expert4 = nn.Linear(10, 20)
        self.gather_router_0 = GatherRouter(dst_num=dst_num)
        self.gather_router_1 = GatherRouter(dst_num=dst_num)

    def forward(self, x, y):
        route_results_x = self.scatter_router_0(x)
        route_results_y = self.scatter_router_1(y)
        x_0 = self.expert1(route_results_x[0])
        x_1 = self.expert2(route_results_x[1])
        y_0 = self.expert3(route_results_y[0])
        y_1 = self.expert4(route_results_y[1])
        x = self.gather_router_0([x_0, y_0])
        y = self.gather_router_1([x_1, y_1])
        return x, y


dy_model = DynamicRouting(2)

x = torch.randn((2, 10))
y = torch.randn((2, 10))

x, y = dy_model(x, y)

print(x)
print(y)


tensor([[ 0.0051,  0.0605, -0.0592,  0.1375,  0.0241,  0.1902,  0.0502, -0.2667,
         -0.0570, -0.1436],
        [ 0.1365,  0.0330,  0.2277,  0.2845, -0.2512,  0.4982,  0.2820, -0.1979,
          0.0123,  0.0257]], grad_fn=<AliasBackward0>)
ProtoTensor(
data: tensor([[ 0.2542,  0.1385,  0.2069,  0.2163, -0.0799,  0.0771, -0.3654,  0.0262,
          0.0798,  0.2070, -0.4161, -0.0174,  0.2717, -0.0166,  0.0447,  0.1913,
          0.0356, -0.2541,  0.2141,  0.0365]], grad_fn=<AliasBackward0>)
tag_stack: [tensor([[0]])]
load stack: [2])
